# Imports

In [ ]:
from itertools import combinations
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Literal, Optional

from matplotlib import rcParams

In [ ]:
# increase figure size to (8, 8)
rcParams["figure.figsize"] = (10, 6)
rcParams["figure.dpi"] = 150
plt.style.use("ggplot")